In [1]:
import pandas as pd
import numpy as np
import os
import fasttext
import json
from sklearn.metrics import f1_score, classification_report
from sklearn.model_selection import train_test_split

PATH_TO_DATASETS = "../datasets"
PATH_TO_POLEMO_CONLL = "../datasets/polemo/dataset_conll"

In [2]:
files = {
    "train": os.path.join(PATH_TO_POLEMO_CONLL,"all.sentence.train_processed.csv"),
    "dev": os.path.join(PATH_TO_POLEMO_CONLL,"all.sentence.dev_processed.csv"),
    "test": os.path.join(PATH_TO_POLEMO_CONLL,"all.sentence.test_processed.csv"),
    "annotation": os.path.join(PATH_TO_DATASETS, "sentiment_data", "political_tweets_annotations.csv")
}

In [3]:
with open(os.path.join("..", "datasets", "emojis.json"), encoding="utf-8") as f:
    emoji_mapping = json.load(f)

emoji_mapping_items = emoji_mapping.items()
def emoji2text_tweet(tweet: str) -> str:
    text = tweet
    for emoji, emoji_text in emoji_mapping_items:
        text = text.replace(emoji, f"<{emoji_text}>")
    return text

In [4]:
def remove_quotes_from_saved_file(txt_path: str):
    text = ""
    with open(txt_path, "r", encoding="utf-8") as f:
        for line in f:
            if line[0] == "\"" and line[-2] == "\"":
                line = line[1:]
                line = line[:-2] + "\n"
            text += line

    os.remove(txt_path)

    with open(txt_path, "w", encoding="utf-8") as f:
        f.write(text)

In [5]:
data_for_fasttext = {}
for dataset, file_path in files.items():
    df = pd.read_csv(file_path)
    df = df[['label','text']]
    df['label'] = "__label__" +  df['label']
    df['text'] = df['text'].apply(emoji2text_tweet)
    df['text'] = df['text'].apply(lambda string: string.lower())
    df['text'] = df['text'].apply(lambda string: string.replace("#",""))
    df['row'] = df['label'] + " " + df['text']
    path = os.path.join(PATH_TO_DATASETS, "sentiment_data", f"{dataset}_data.txt")
    df['row'].to_csv(path, index=False, header=False)
    remove_quotes_from_saved_file(path)
    data_for_fasttext[dataset] = {}
    data_for_fasttext[dataset]["labels"] = list(df['label'].values)
    data_for_fasttext[dataset]["texts"] = list(df['text'].values)
    data_for_fasttext[dataset]["dataframe"] = df

In [6]:
tweets_data = data_for_fasttext['annotation']
texts_train_val, texts_test, labels_train_val, labels_test = train_test_split(tweets_data['texts'], tweets_data['labels'], test_size=0.1, random_state=42)
texts_train, texts_val, labels_train, labels_val = train_test_split(texts_train_val, labels_train_val, test_size=1/9, random_state=42)

train_polemo = data_for_fasttext['train']['dataframe'][["text", "label"]]
val_polemo = data_for_fasttext['dev']['dataframe'][["text", "label"]]
test_polemo = data_for_fasttext['test']['dataframe'][["text", "label"]]

train_tweets = pd.DataFrame(data={"text": texts_train, "label": labels_train})
val_tweets = pd.DataFrame(data={"text": texts_val, "label": labels_val})
test_tweets = pd.DataFrame(data={"text": texts_test, "label": labels_test})

train = train_polemo.append(train_tweets)
val = val_polemo.append(val_tweets)
test = test_polemo.append(test_tweets)

train['row'] = train['label'] + " " + train['text']
val['row'] = val['label'] + " " + val['text']
test['row'] = test['label'] + " " + test['text']

train['row'].to_csv(os.path.join(PATH_TO_DATASETS, "sentiment_data", f"full_train_data.txt"), index=False, header=False)
val['row'].to_csv(os.path.join(PATH_TO_DATASETS, "sentiment_data", f"full_val_data.txt"), index=False, header=False)
test['row'].to_csv(os.path.join(PATH_TO_DATASETS, "sentiment_data", f"full_test_data.txt"), index=False, header=False)
remove_quotes_from_saved_file(os.path.join(PATH_TO_DATASETS, "sentiment_data", f"full_train_data.txt"))
remove_quotes_from_saved_file(os.path.join(PATH_TO_DATASETS, "sentiment_data", f"full_val_data.txt"))
remove_quotes_from_saved_file(os.path.join(PATH_TO_DATASETS, "sentiment_data", f"full_test_data.txt"))

In [7]:
print("Training classifier only on PolEmo training data...")
model =fasttext.train_supervised(input=os.path.join(PATH_TO_DATASETS, "sentiment_data", "train_data.txt"), wordNgrams=1, neg=5,dim=300, lr=0.005, epoch=500, loss="ns", verbose=1, label_prefix='__label__')
test_results = model.predict(data_for_fasttext["test"]["texts"])
annotation_results = model.predict(data_for_fasttext["annotation"]["texts"])

print(f"F1-score for all tweets: {f1_score(annotation_results[0],data_for_fasttext['annotation']['labels'],average='macro')}")
print(f"F1-score for test set of PolEmo: {f1_score(test_results[0],data_for_fasttext['test']['labels'],average='macro')}")
print()

print("Training classifier on PolEmo training data and 80% of political tweets")
model =fasttext.train_supervised(input=os.path.join(PATH_TO_DATASETS, "sentiment_data", "full_train_data.txt"), wordNgrams=1, neg=5,dim=300, lr=0.005, epoch=500, loss="ns", verbose=1, label_prefix='__label__')
test_results = model.predict(list(test_polemo["text"].values))
annotation_results = model.predict(list(test_tweets['text'].values))

print(f"F1-score for test set of political tweets: {f1_score(annotation_results[0],list(test_tweets['label'].values),average='macro')}")
print(f"F1-score for test set of PolEmo: {f1_score(test_results[0],list(test_polemo['label'].values),average='macro')}")
print()

Training classifier only on PolEmo training data...
F1-score for all tweets: 0.29374087451331876
F1-score for test set of PolEmo: 0.5863994076552392

Training classifier on PolEmo training data and 80% of political tweets
F1-score for test set of political tweets: 0.3604167498303461
F1-score for test set of PolEmo: 0.5882343321594454



In [ ]:
dims = []
ngrams = []
losses = []
negs = []
tweets_f1_scores = []
dev_f1_scores = []
for dim in [300, 500]:
    for ngram in [1,2,3,4,5]:
        for method in ["hs", "ns", "softmax"]:
            for neg in [5,10,15,20]:
                model =fasttext.train_supervised(input=os.path.join(PATH_TO_DATASETS, "sentiment_data", "full_train_data.txt"), wordNgrams=ngram, neg=neg,dim=dim, lr=0.005, epoch=500, loss=method, verbose=1, label_prefix='__label__')
                dev_results = model.predict(list(val_polemo["text"].values))
                annotation_results = model.predict(list(val_tweets["text"].values))
                dims.append(dim)
                ngrams.append(ngram)
                losses.append(method)
                negs.append(neg)
                tweets_f1_scores.append(f1_score(annotation_results[0],list(val_tweets['label'].values),average='macro'))
                dev_f1_scores.append(f1_score(dev_results[0],list(val_polemo['label'].values),average='macro'))
                print(f"Loss method - {method}")
                print(f"Dim - {dim}")
                print(f"Ngram - {ngram}")
                print(f"Negative samples - {neg}")
                print(f"F1-score for all tweets: {tweets_f1_scores[-1]}")
                print(f"F1-score for dev set: {dev_f1_scores[-1]}")
                print()

results = pd.DataFrame(data={"dim": dims,
                             "ngram" : ngrams,
                             "loss": losses,
                             "neg": negs,
                             "tweets_f1_score": tweets_f1_scores,
                             "dev_f1_score": dev_f1_scores})

results.to_csv(os.path.join("..","reports","sentiment_classification_results.csv"))

In [ ]:
lrs = []
epochs = []
tweets_f1_scores = []
dev_f1_scores = []
for lr in [0.001, 0.005, 0.0001, 0.0005]:
    for epoch in [100,250,500,1000]:
        model =fasttext.train_supervised(input=os.path.join(PATH_TO_DATASETS, "sentiment_data", "full_train_data.txt"), wordNgrams=5, neg=5,dim=300, lr=lr, epoch=epoch, loss="ns", verbose=1, label_prefix='__label__')
        dev_results = model.predict(list(val_polemo["text"].values))
        annotation_results = model.predict(list(val_tweets["text"].values))
        lrs.append(lr)
        epochs.append(epoch)
        tweets_f1_scores.append(f1_score(annotation_results[0],list(val_tweets['label'].values),average='macro'))
        dev_f1_scores.append(f1_score(dev_results[0],list(val_polemo['label'].values),average='macro'))
        print(f"Learning rate - {lr}")
        print(f"Epochs - {epoch}")
        print(f"F1-score for all tweets: {tweets_f1_scores[-1]}")
        print(f"F1-score for dev set: {dev_f1_scores[-1]}")
        print()

In [28]:
lr_epoch_results = pd.DataFrame(data={"lr": lrs,
                             "epoch" : epochs,
                             "tweets_f1_score": tweets_f1_scores,
                             "dev_f1_score": dev_f1_scores})

lr_epoch_results.to_csv(os.path.join("..","reports","sentiment_classification_lr_epoch_results.csv"), index=False)

In [8]:
model = fasttext.train_supervised(input=os.path.join(PATH_TO_DATASETS, "sentiment_data", "full_train_data.txt"), wordNgrams=5, neg=5,dim=300, lr=0.005, epoch=500, loss="ns", verbose=1, label_prefix='__label__')
test_results = model.predict(list(test_polemo["text"].values))
annotation_results = model.predict(list(test_tweets["text"].values))


In [9]:
print(f"F1-score for all tweets: {f1_score(annotation_results[0],list(test_tweets['label'].values),average='macro')}")
print(f"F1-score for dev set: {f1_score(test_results[0],list(test_polemo['label'].values),average='macro')}")
print()

F1-score for all tweets: 0.42483660130718953
F1-score for dev set: 0.6493587265781213



In [10]:
print("Classification report for political tweets")
print(classification_report(list(test_tweets['label'].values), annotation_results[0]))

print("Classification report for polemo data")
print(classification_report(list(test_polemo['label'].values), test_results[0]))


Classification report for political tweets
                    precision    recall  f1-score   support

__label__ambiguous       0.00      0.00      0.00         5
 __label__negative       0.56      0.58      0.57        24
  __label__neutral       0.46      0.65      0.54        26
 __label__positive       0.69      0.51      0.59        49

          accuracy                           0.54       104
         macro avg       0.43      0.44      0.42       104
      weighted avg       0.57      0.54      0.54       104

Classification report for polemo data
                    precision    recall  f1-score   support

__label__ambiguous       0.55      0.33      0.41       681
 __label__negative       0.69      0.81      0.74      2123
  __label__neutral       0.71      0.70      0.70      1419
 __label__positive       0.76      0.71      0.74      1522

          accuracy                           0.70      5745
         macro avg       0.68      0.64      0.65      5745
      weighted

In [ ]:
test_texts = list(test_tweets["text"].values)
test_labels = list(test_tweets["label"].values)
print("Tweets which were incorrrectly predicted:")
for i in range(len(test_texts)):
    pred = model.predict([test_texts[i]])
    if test_labels[i] != pred[0][0][0]:
        print(f"Tweet text: {test_texts[i]}")
        print(f"\t True label: {test_labels[i]}, predicted label: {pred[0][0][0]}, probability: {pred[1][0][0]}")
        print()